# 02 - Data from the Web

We start importing all the libraries that we are going to use.

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup # if you don't have this library, 
# install it. The homework instructions tell us to use it.


Generating the base of our futures requests

In [3]:
payloadform = {'ww_i_reportModel': '133685247'}
isa_baseurl = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS'
isa_formurl= isa_baseurl + ".filter"
r = requests.get(isa_formurl, params=payloadform)
s = r.url + " " + str(r.status_code)
s

'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247 200'

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')
inputs = soup.select('form input') # extraemos una lista con todos los 
# inputs del form ,los 4 primeros inputs no nos valen
inputs[5]['name'] # nombre de la opcion
tags=inputs[6].select('option') # extraemos las posibles opcicones 
# de los inputs 
tags[1].contents # mostramos el contenido o texto dentro de una opcion
tags[1]['value'] # valor que meter en nuestra petición rest (de cada opcion)
inputs[5].select('option')[9].contents[0] # extraemos los nombres de los inputs

'Informatique'

With the data that we have just obtain, we will make the request petitions with a loops


In [52]:
payloadfix = {'ww_x_GPS': '-1', 'ww_i_reportModel': '133685247', 'ww_i_reportModelXsl': '133685270', 'ww_x_UNITE_ACAD': '','ww_x_PERIODE_ACAD': '', 'ww_x_PERIODE_PEDAGO': '','ww_x_HIVERETE': '' }
isa_dataurl = isa_baseurl + ".html"
# We have to make a loop here changing the payloadfix values. (only the next ones)
payloadfix['ww_x_UNITE_ACAD'] = inputs[5].select('option')[9]['value']
#payloadfix['ww_x_PERIODE_ACAD'] = inputs[6].select('option')[10]['value']#decrement [10]==2007
#payloadfix['ww_x_PERIODE_PEDAGO'] = inputs[7].select('option')[1]['value']#[7]==bach6
#payloadfix
#r = requests.get(isa_dataurl, params=payloadfix)
#r.url


In [202]:
# Here we will need to parse the html using BeautifulSoup library.
# The data that we want comes in a html table, won't be really hard to parse it.
#Data = BeautifulSoup(r.text, 'html.parser') 


def extractTable(year,sem):
    payloadfix['ww_x_PERIODE_ACAD'] = inputs[6].select('option')[year]['value']
    payloadfix['ww_x_PERIODE_PEDAGO'] = inputs[7].select('option')[sem]['value']#[7]==bach6
    if(sem==1):
        payloadfix['ww_x_HIVERETE'] = inputs[8].select('option')[1]['value']
    else:
        payloadfix['ww_x_HIVERETE'] = inputs[8].select('option')[2]['value']
    
    r = requests.get(isa_dataurl, params=payloadfix)
    Data = BeautifulSoup(r.text, 'html.parser') 
    """Extract the table from HTML into a pandas Dataframe."""
    BStable = BeautifulSoup(r.text, 'html.parser').find('table')
    
    BSRows = BStable.findAll('tr')
    # BSrows[0] is just the title of the table, so we ignore it.
    headerBSRow = BSRows[1]
    headers = [th.string for th in headerBSRow]
    # We drop the last column, because it's always empty (originally it is used to make the HTML table look better).
    rows = [[tr.string for tr in BSRow.findAll('td')[:-1]] for BSRow in BSRows[2:]]
    return pd.DataFrame(data=rows, columns=headers)[headers[-1]]
year=10
semester=1
data=[]
i=0
while(year>0):
#    if(semester==1 and year<3):
#        semester=7
#        continue
    course=extractTable(year,semester)
    course.name=str(i)
    data.append(course)
    if(semester==1):
        semester=7
    else:
        year-=1
        semester=1
    i+=1
n=0
df_beg={}
df_end={}
index=0
print('creating df')
for i in data:
    if(index%2==0):
        if(index/2>8):
            index+=1
            continue
        df_beg[str(int(index/2))]=i
        print('num:',int(index/2),' len:',len(i))
    else:
        df_end[str(int((index+1)/2-1))]=i
    index+=1
index=0#i current semester[0..7]
index2=1#j next semester[1..7] for bachelor1
aux=0
#NOTE: we know all the students can enroll to the bach1, and then pass to the 2nd bach or not.
#and you're supposed to enroll to the same bachelor the year after
#but failing on the bach1 does not imply that someone's gonna enroll to the next year.
#you can enroll two years after failing too if you're a mess, your money's always welcome.
#so, it's the same as passing, but we can't figured out if you really pass.Just knowing your last enrollment.
#we should check the master1 instead of bach6 but it's out of context
while(index<len(df_beg)-1):
    while(index2<len(df_beg)):
        for i in df_beg[str(index2)].values:
            if i in df_beg[str(index)].values:
                #print(i)
                found=df_beg[str(index2)][df_beg[str(index2)]==i] #give us the subserie found
                found=found.index[0]#get index from serie
                df_beg[str(index2)]=df_beg[str(index2)].drop(found)#drop repeated student(he was enrolled the previous year)
        index2+=1
    index+=1
    index2=index+1
df_beg=pd.DataFrame(df_beg)
df_end=pd.DataFrame(df_end)
print('ending df')
index=0#for bachelor1 dataframe
index2=2#for bachelor6 dataframe
#while(index<len(df_beg) and index2<len(df_end)):
#print(type(df_beg[str(index)]))
three_aux=0
three_yrs_tot=0
three_yrs=[]#we're gonna fetch (beginyear,endyear,total)
four_aux=0
four_yrs=[]
four_yrs_tot=0
five_aux=0
five_yrs=[]
five_yrs_tot=0
six_aux=0
six_yrs=[]
six_yrs_tot=0
while(index<len(df_beg)):
    if(index2==10):
        index+=1
        index2=index+2
    if(index2==10):
        break
    years=index2-index+1
    for i in df_end[str(index2)].values:
        if i in df_beg[str(index)].values:
            if years==3:        
                three_yrs_tot+=1
                three_aux+=1
            elif years==4:
                four_yrs_tot+=1
                four_aux+=1
            elif years==5:
                five_yrs_tot+=1
                five_aux+=1
            elif years==6:
                six_yrs_tot+=1
                six_aux+=1
    if years==3:
        three_yrs.append([index,index2,three_aux])
        three_aux=0
    elif years==4:
        four_yrs.append([index,index2,four_aux])
        four_aux=0
    elif years==5:
        five_yrs.append([index,index2,five_aux])
        five_aux=0
    elif years==6:
        six_yrs.append([index,index2,six_aux])
        six_aux=0
    index2+=1
print("TOTAL: 3 years: ",three_yrs_tot,"\t4 years: ",four_yrs_tot,"\t5 years:",five_yrs_tot,"\t6 years:",six_yrs_tot)
print('3 years',three_yrs)
print('4 years',four_yrs)
print('5 years',five_yrs)
print('6 years',six_yrs)

#a=set(df_beg[str(index)].values)
#b=set(df_end[str(index)].values)
#c= a&b
#print(c)
#print(df_beg[str(index)].values&df_end[str(index2)].values)
    
#if '179383' in df_beg['1'].values:
#    print('existe fuaaa')
#    df_beg=df_beg[df_beg['1']!='179383']
#print(len(df_beg['1']))

#df_end
#type(extractTable(year,semester))
#print(data)
#extractTable(year,semester).values[0]
#print(Data.html.body.table('No Sciper'))
#table = Data.html.body.table
#i=10# elementos td del html
#years=10
#sciphers_bach1_years=[]
#sciphers_bach6_years=[]
#sciphers=[]
#semester=1
#n=0#sciphersrows
#print('jjjj')

#while(years>0):
#    payloadfix['ww_x_PERIODE_ACAD'] = inputs[6].select('option')[years]['value']#decrement [10]==2007
#    payloadfix['ww_x_PERIODE_PEDAGO'] = inputs[7].select('option')[semester]['value']#[7]==bach6
#    r = requests.get(isa_dataurl, params=payloadfix)
#    Data = BeautifulSoup(r.text, 'html.parser') 
#    table = Data.html.body.table
#
#    for indX,row in enumerate(table.find_all('td')):
#        if(indX==i):
#            sciphers.insert(n,row.text)
#            i+=12
#            n+=1
#    if(semester==1):
#        sciphers_bach1_years.insert(abs(years-10),sciphers)
#        years-=1
#        semester=1
#    else: 
#        sciphers_bach6_years.insert(abs(years-10),sciphers)
#        semester=1
#    n=0
#    i=10
#    sciphers=[]
    
#print(sciphers_years[9])
#sc_bach1={'No Scipher_2007_Bach1':pd.Series(sciphers_bach1_years[0]),'No Scipher_2008_Bach1':pd.Series(sciphers_bach1_years[1]),
#   'No Scipher_2009_Bach1':pd.Series(sciphers_bach1_years[2]),'No Scipher_2010_Bach1':pd.Series(sciphers_bach1_years[3]),
#   'No Scipher_2011_Bach1':pd.Series(sciphers_bach1_years[4]),'No Scipher_2012_Bach1':pd.Series(sciphers_bach1_years[5]),
#   'No Scipher_2013_Bach1':pd.Series(sciphers_bach1_years[6]),'No Scipher_2014_Bach1':pd.Series(sciphers_bach1_years[7]),
#   'No Scipher_2015_Bach1':pd.Series(sciphers_bach1_years[8]),'No Scipher_2016_Bach1':pd.Series(sciphers_bach1_years[9])}
#print('jjjj')
#sc_bach6={'No Scipher_2007_Bach6':pd.Series(sciphers_bach6_years[0]),'No Scipher_2008_Bach6':pd.Series(sciphers_bach6_years[1]),
#   'No Scipher_2009_Bach6':pd.Series(sciphers_bach6_years[2]),'No Scipher_2010_Bach6':pd.Series(sciphers_bach6_years[3]),
#   'No Scipher_2011_Bach6':pd.Series(sciphers_bach6_years[4]),'No Scipher_2012_Bach6':pd.Series(sciphers_bach6_years[5]),
#   'No Scipher_2013_Bach6':pd.Series(sciphers_bach6_years[6]),'No Scipher_2014_Bach6':pd.Series(sciphers_bach6_years[7]),
#   'No Scipher_2015_Bach6':pd.Series(sciphers_bach6_years[8]),'No Scipher_2016_Bach6':pd.Series(sciphers_bach6_years[9])}
#print(sciphers_bach1_years[0])
#bach1=pd.DataFrame(sc_bach1)
#bach6=pd.DataFrame(sc_bach6)
#bach6

creating df
num: 0  len: 90
num: 1  len: 96
num: 2  len: 117
num: 3  len: 153
num: 4  len: 166
num: 5  len: 198
num: 6  len: 206
num: 7  len: 242
num: 8  len: 216
ending df
TOTAL: 3 years:  327 	4 years:  134 	5 years: 40 	6 years: 6
3 years [[0, 2, 37], [1, 3, 35], [2, 4, 31], [3, 5, 46], [4, 6, 42], [5, 7, 68], [6, 8, 47], [7, 9, 21]]
4 years [[0, 3, 13], [1, 4, 15], [2, 5, 21], [3, 6, 23], [4, 7, 28], [5, 8, 33], [6, 9, 1]]
5 years [[0, 4, 2], [1, 5, 6], [2, 6, 12], [3, 7, 6], [4, 8, 13], [5, 9, 1]]
6 years [[0, 5, 1], [1, 6, 1], [2, 7, 3], [3, 8, 1], [4, 9, 0]]
